# High-level RNN TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params_lstm import *
from common.utils import *

/opt/conda/envs/py3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Numpy:  1.13.3
Tensorflow:  1.4.0
GPU:  ['Tesla K80']


In [3]:
def create_symbol(CUDNN=True):
    word_vectors = tf.contrib.layers.embed_sequence(X, vocab_size=MAXFEATURES, embed_dim=EMBEDSIZE)
    word_list = tf.unstack(word_vectors, axis=1)
    
    if not CUDNN:
        cell = tf.contrib.rnn.GRUCell(NUMHIDDEN)
        outputs, states = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)
    else:
        # Using cuDNN since vanilla RNN
        cudnn_cell = tf.contrib.cudnn_rnn.CudnnGRU(num_layers=1, 
                                                   num_units=NUMHIDDEN, 
                                                   input_size=EMBEDSIZE)
        params_size_t = cudnn_cell.params_size()
        params = tf.Variable(tf.random_uniform([params_size_t], -0.1, 0.1), validate_shape=False)   
        input_h = tf.Variable(tf.zeros([1, BATCHSIZE, NUMHIDDEN]))
        outputs, states = cudnn_cell(input_data=word_list,
                                     input_h=input_h,
                                     params=params)
        logits = tf.layers.dense(outputs[-1], 2, activation=None, name='output')
    return logits

In [4]:
def init_model(m):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(LR, BETA_1, BETA_2, EPS)
    training_op = optimizer.minimize(loss)
    return training_op

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 5.01 s, sys: 312 ms, total: 5.32 s
Wall time: 5.33 s


In [6]:
%%time
# Place-holders
X = tf.placeholder(tf.int32, shape=[None, MAXLEN])
y = tf.placeholder(tf.int32, shape=[None])
sym = create_symbol()

CPU times: user 467 ms, sys: 55.7 ms, total: 522 ms
Wall time: 522 ms


In [7]:
%%time
model = init_model(sym)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

CPU times: user 1.17 s, sys: 521 ms, total: 1.69 s
Wall time: 1.69 s


In [8]:
%%time
# 30s
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.90625
1 Train accuracy: 0.953125
2 Train accuracy: 1.0
CPU times: user 20.6 s, sys: 6.29 s, total: 26.9 s
Wall time: 30.4 s


In [9]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym, 1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 10.3 s, sys: 3.23 s, total: 13.5 s
Wall time: 13.2 s


In [10]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.857532051282
